# Generación de inferencias

Este notebook esta contruido de tal forma unicamente necesita las carpetas y archivos generados por el notebook anterior.

Establecemos el nombre del archivo que contiene nuestros datos .data sin la extensión, esto para poder buscar el dataset y definir nombres de archivos.

In [ ]:
dataset= "input_bo222-200k-1173k"

Importamos los modulos necesarios

In [ ]:
import numpy as np
import os
import shutil
import torch
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import re
from numpy import linalg as la
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde

Creamos las carpetas que necesitamos, de existir las eliminamos y las volvemos a crear.

In [ ]:
carpetas = ["input-graficas", "output-graficas", "inferencias","temp"]
carpeta_principal = "graficas-inferencias"

def crear_carpeta(carpeta):
    ruta_carpeta = os.path.join(carpeta_principal, carpeta)
    os.makedirs(ruta_carpeta)
    print(f"La carpeta '{ruta_carpeta}/' ha sido creada.")

# Verificar y eliminar la carpeta principal si existe
if os.path.exists(carpeta_principal):
    shutil.rmtree(carpeta_principal)
    print(f"La carpeta '{carpeta_principal}/' existía y ha sido eliminada junto con su contenido.")

# Crear la carpeta principal de nuevo
os.makedirs(carpeta_principal)
print(f"La carpeta '{carpeta_principal}/' ha sido creada de nuevo.")

# Crear las carpetas dentro de la carpeta principal
for carpeta in carpetas:
    crear_carpeta(carpeta)

Cargamos los archivos que utilizaremos: modelo entrenado, dataset utilizado, indices de los conjutos de datos (train, validation, test)

In [ ]:
# Buscamos el modelo entrenado con el número de epocas mas alto
nombre_max_epoch = max((f for f in os.listdir('output') if re.match(r'epoch=(\d+)-val_loss', f)),
                       key=lambda x: int(re.search(r'epoch=(\d+)-val_loss', x).group(1)),
                       default=None)

# Ruta al modelo entrenado
path_trained_model=f"output/{nombre_max_epoch}"

# Cargamos el modelo entrenado
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True,device="cuda")

# Cargar archivo del dataset que utilizo torchmd-net
dataset_h5 = h5py.File(f"construccion-dataset/procesados/{dataset}-f.h5", 'r')

# Cargar archivo splits
splits = np.load("output/splits.npz")
idx_test = np.sort(splits['idx_test'])

Definimos el dispositivo donde vamos a hacer las operaciones

In [ ]:
device = torch.device("cuda:0")

Seleccionamos el conjunto de datos test

In [ ]:
pos_test=dataset_h5["222_atoms"]['pos'][idx_test]
types_test=dataset_h5["222_atoms"]["types"][idx_test]
ref_energy_test=dataset_h5["222_atoms"]["energy"][idx_test]
ref_forces_test=dataset_h5["222_atoms"]["forces"][idx_test]

Convertimos nuestros datos en unos que puedan ser utilizados por el modelo de inferencias y los guardamos en donde vamos a hacer operaciones

In [ ]:
pos_sample = torch.tensor(pos_test, dtype=torch.float32, device=device)
z_sample = torch.tensor(types_test, dtype=torch.long, device=device)

Calculamos las ainferencias y las guardamos en archivos .data

In [ ]:
for sample in range(len(ref_energy_test)):
    # Hacer la inferencia
    n_energy_inferred, n_forces_inferred = model(z_sample[sample], pos_sample[sample])
    
    # Convertir los datos a cadenas
    n_energy_str = str(n_energy_inferred.item())
    n_forces_str = '\n'.join([' '.join(map(str, fila)) for fila in n_forces_inferred.tolist()])
    # Guardar los datos en archivos
    with open(f'graficas-inferencias/temp/inf_energias_test.dat', 'a') as file_energy:
        file_energy.write(n_energy_str+'\n')

    with open(f'graficas-inferencias/temp/inf_fuerzas_test.dat', 'a') as file_forces:
        file_forces.write(n_forces_str+'\n')

Cargar las inferencias de los archivos .data en numpy arrays y redimensionarlos

In [ ]:
# Cargar y remodelar los datos de energía de prueba
inf_energy_test = np.loadtxt("graficas-inferencias/temp/inf_energias_test.dat")
inf_energy_test = np.reshape(inf_energy_test, len(ref_energy_test))

# Cargar y remodelar los datos de fuerza de prueba
inf_forces_test = np.loadtxt("graficas-inferencias/temp/inf_fuerzas_test.dat")
inf_forces_test = np.reshape(inf_forces_test, (len(ref_energy_test), len(types_test[0]), 3))

Guardar inferencias en archivo .h5

In [ ]:
filename = f"graficas-inferencias/inferencias/{dataset}-inferencias.h5"

f=h5py.File(filename, "w")

group_name = "test"
group = f.create_group(group_name)
# Guardar datos en el grupo
group.create_dataset("idx", data=idx_test)
group.create_dataset("energy_inf", data=inf_energy_test)
group.create_dataset("forces_inf", data=inf_forces_test)
group.create_dataset("energy_ref", data=ref_energy_test)
group.create_dataset("forces_ref", data=ref_forces_test)
group.create_dataset("pos", data=pos_test)
group.create_dataset("types", data=types_test)
f.close()

In [ ]:
dataset_h5.close()

# Contraste archivo de entrenamiento .h5  y archivo de inferencias .h5

Funciones a utilizar

In [ ]:
# Funcion para intercambiar el elemento atomico por su numero atomico
def periodic_table(input_elemento):
    tabla_periodica = {"H": 1, "O": 8, "Al": 13}

    if isinstance(input_elemento, str):
        # Si se proporciona el símbolo del elemento
        elemento = input_elemento.capitalize()
        if elemento in tabla_periodica:
            return tabla_periodica[elemento]
        else:
            return None
    elif isinstance(input_elemento, int):
        # Si se proporciona el número atómico
        for simbolo, numero_atomico in tabla_periodica.items():
            if numero_atomico == input_elemento:
                return simbolo
        # Si no se encuentra el número atómico en la tabla
        return None
    else:
        # Si el tipo de entrada no es válido
        return None

def imprimir_muestra_h5(ruta_archivo_h5, grupo, datos, muestra):
    archivo_h5=h5py.File(ruta_archivo_h5, "r")
    
    idx_test = archivo_h5[grupo]["idx"]
    
    if datos == "ref":
        arrays_h5=["energy_ref","types","pos","forces_ref"]
    if datos == "inf":
        arrays_h5=["energy_inf","types","pos","forces_inf"]
    
    
    muestra=idx_test[muestra]
    grupo_actual = archivo_h5[grupo]
    num_atoms_frame = len(grupo_actual["types"][muestra])
    
    frame = f"{num_atoms_frame}\n"
    frame = frame + f"MD_Step: {muestra} Total_energy = {grupo_actual[arrays_h5[0]][muestra]:.10f}\n"
    for i in range(num_atoms_frame):
    
        line_type = periodic_table(int(grupo_actual[arrays_h5[1]][muestra][i]))
        line_x = grupo_actual[arrays_h5[2]][muestra][i][0]
        line_y = grupo_actual[arrays_h5[2]][muestra][i][1]
        line_z = grupo_actual[arrays_h5[2]][muestra][i][2]
        line_fx = grupo_actual[arrays_h5[3]][muestra][i][0]
        line_fy = grupo_actual[arrays_h5[3]][muestra][i][1]
        line_fz = grupo_actual[arrays_h5[3]][muestra][i][2]
    
        # Formatea la cadena con manejo de valores None
        linea = "{:>2}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}{:>20.10f}".format(
            line_type,
            line_x, 
            line_y, 
            line_z,
            line_fx, 
            line_fy, 
            line_fz
        )
        frame = frame + linea + "\n"
    print(frame)
    archivo_h5.close()

Muestra a imprimir

In [ ]:
muestra=20

Imprimir muestra de referencia

In [ ]:
imprimir_muestra_h5(f"graficas-inferencias/inferencias/{dataset}-inferencias.h5", "test", "inf", muestra)

Imprimir muestra de inferencia

In [ ]:
imprimir_muestra_h5(f"graficas-inferencias/inferencias/{dataset}-inferencias.h5", "test", "ref", muestra)

# Procesamiento de datos para graficar

Ya tenemos las inferencias generadas por el modelo entrenado. Para hacer un analisis mas profundo procesamos los datos de referencia y los de inferencia.

Funciones a utilizar

In [ ]:
def angle_forces(a,b):
    from math import acos, degrees
    import numpy as np
    from numpy import linalg as la
    theta_degrees=degrees(acos((np.dot(a,b))/(la.norm(a)*la.norm(b))))
    return theta_degrees

Cargar archivo de inferencias

In [ ]:
#Cargar los arreglos del archivo de inferencias
archivo_inferencias=h5py.File(f"graficas-inferencias/inferencias/{dataset}-inferencias.h5", "r")
test=archivo_inferencias["test"]
types_test=test["types"]
forces_test_inf=test["forces_inf"]
forces_test_ref=test["forces_ref"]
energy_test_inf=test["energy_inf"]
energy_test_ref=test["energy_ref"]

## Procesar el archivo de inferencias para graficar

Cargamos la información del entrenamiento y multiplicamos por 1000 para adecuar la escala al estandar.

In [ ]:
# Lee el archivo CSV en un DataFrame de pandas
path_metric_csv="output/metrics.csv"
metrics = pd.read_csv(path_metric_csv)

# Multiplica los valores por 1000 en las columnas train_loss y val_loss
#metrics['train_loss'] *= 1000.0/222
#metrics['val_loss'] *= 1000.0/222

# Seleccionar las columnas deseadas
selected_columns = ['epoch', 'train_loss', 'val_loss']
train_metrics = metrics[selected_columns]

# Guardar en un archivo .data con el encabezado deseado
with open("graficas-inferencias/input-graficas/epoch_vs_loss.data", 'w') as file:
    file.write("#Epoch MAE_Train MAE_Val\n")
    train_metrics.to_csv(file, sep=' ', index=False, header=False, float_format='%.6f')

Calculamos el ángulo entre las fuerzas de referencia e inferencia y creamos un archivo para cada gráfica.

In [ ]:
atomos_unicos = np.unique(types_test).tolist()

for i, molecula in enumerate(types_test):
    for atomo in atomos_unicos:
        indices_atomo = np.where(molecula == atomo)[0]
        for k in indices_atomo:
            force_reference=forces_test_ref[i][k]
            force_inferred=forces_test_inf[i][k]
            angle=angle_forces(force_reference,force_inferred)
            atomo=int(atomo)
            with open(f"graficas-inferencias/input-graficas/modules-test-{atomo}.dat","a") as archivo:
                archivo.write(str(la.norm(force_reference))+"\t"+str(la.norm(force_inferred))+"\n")
            with open(f"graficas-inferencias/input-graficas/angles-test-{atomo}.dat","a") as archivo:
                archivo.write(str(angle)+"\n")
    with open(f"graficas-inferencias/input-graficas/energy_test_inf.dat","a") as archivo:
        archivo.write(str(energy_test_inf[i])+"\n")
    with open(f"graficas-inferencias/input-graficas/energy_test_sam.dat","a") as archivo:
        archivo.write(str(energy_test_ref[i])+"\n")
    with open(f"graficas-inferencias/input-graficas/energy_abs_dif.dat","a") as archivo:
        archivo.write(str(np.abs(energy_test_ref[i]-energy_test_inf[i])/222)+"\n")

#Cerrar correctamente el archivo
archivo_inferencias.close()

# Graficas

## Epoch vs loss

In [ ]:
# Leer el archivo .data
path_data_file = "graficas-inferencias/input-graficas/epoch_vs_loss.data"
data = pd.read_csv(path_data_file, sep=' ', comment='#', header=None, names=['Epoch', 'MAE_Train', 'MAE_Test'])

# Graficar
plt.plot(data['Epoch'], data['MAE_Train']*1000.0/222, label='MAE_Train')
plt.plot(data['Epoch'], data['MAE_Test']*1000.0/222, label='MAE_Val')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error (meV)')
plt.title('Epoch vs Mean Squared Error')
plt.legend()
plt.ylim(0, 6)  # Establecer el rango del eje y
plt.grid(True)
plt.savefig("graficas-inferencias/output-graficas/figura-01.png")
plt.show()


# Encontrar los mínimos de MAE_Train y MAE_Test
print("Mínimo de MAE_Train:", data['MAE_Train'].min())
print("Mínimo de MAE_Test:", data['MAE_Test'].min())

## Energy

In [ ]:
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas
ref_data = pd.read_csv(REF_PATH, header=None, names=['REF_PATH'])/222
inf_data = pd.read_csv(INF_PATH, header=None, names=['INF_PATH'])/222

# Obtener los valores mínimos y máximos de ambos conjuntos de datos
xy_min = min(ref_data['REF_PATH'].min(), inf_data['INF_PATH'].min())
xy_max = max(ref_data['REF_PATH'].max(), inf_data['INF_PATH'].max())

# Graficar
plt.scatter(ref_data['REF_PATH'], inf_data['INF_PATH'])
plt.xlabel('DFT energy (eV)')  # Los datos de referencia deben estar en el eje x
plt.ylabel('TorchMD-net energy (eV)')  # Los datos de inferencia deben estar en el eje y
plt.title('Reference energy vs Inferred energy')
plt.grid(True)

# Establecer los límites de los ejes
plt.xlim(xy_min, xy_max)
plt.ylim(xy_min, xy_max)

plt.savefig("graficas-inferencias/output-graficas/figura-02.png")
plt.show()

#### Discretos

In [ ]:
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas y dividir por 222
ref_data = pd.read_csv(REF_PATH, header=None, names=['REF_PATH']) / 222
inf_data = pd.read_csv(INF_PATH, header=None, names=['INF_PATH']) / 222

# Obtener los valores mínimos y máximos de ambos conjuntos de datos
xy_min = min(ref_data['REF_PATH'].min(), inf_data['INF_PATH'].min())
xy_max = max(ref_data['REF_PATH'].max(), inf_data['INF_PATH'].max())

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data['REF_PATH'], inf_data['INF_PATH'], bins=50, range=[[xy_min, xy_max], [xy_min, xy_max]], density=True)

# Filtrar los datos donde la densidad es cero
non_zero_indices = hist != 0
hist_filtered = np.where(non_zero_indices, hist, np.nan)

# Graficar el histograma 2D de densidad
plt.imshow(hist_filtered.T, extent=[xy_min, xy_max, xy_min, xy_max], cmap='Blues', origin='lower', aspect='auto')
plt.colorbar(label='Density')
plt.xlabel('DFT energy (eV)')
plt.ylabel('TorchMD-net energy (eV)')
plt.title('Reference energy vs Inferred energy (Density)')
plt.grid(True)

plt.savefig("graficas-inferencias/output-graficas/figura-03.png")
plt.show()

In [ ]:
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas y dividir por 222
ref_data = pd.read_csv(REF_PATH, header=None, names=['REF_PATH']) / 222
inf_data = pd.read_csv(INF_PATH, header=None, names=['INF_PATH']) / 222

# Obtener los valores mínimos y máximos de ambos conjuntos de datos
xy_min = min(ref_data['REF_PATH'].min(), inf_data['INF_PATH'].min())
xy_max = max(ref_data['REF_PATH'].max(), inf_data['INF_PATH'].max())

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data['REF_PATH'], inf_data['INF_PATH'], bins=50, range=[[xy_min, xy_max], [xy_min, xy_max]], density=True)

# Filtrar los datos donde la densidad es cero
non_zero_indices = hist != 0
xpos, ypos = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
xpos = xpos[non_zero_indices]
ypos = ypos[non_zero_indices]
dz = hist[non_zero_indices]

# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar la superficie
ax.plot_trisurf(xpos.ravel(), ypos.ravel(), dz.ravel(), cmap='Blues')

# Establecer los límites de los ejes x e y
ax.set_xlim(xy_min, xy_max)
ax.set_ylim(xy_min, xy_max)

# Agregar etiquetas y título
ax.set_xlabel('DFT Energy (eV)')
ax.set_ylabel('TorchMD-Net Energy (eV)')
ax.set_zlabel('Density')
ax.set_title('Reference energy vs Inferred energy (Density Surface)')

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-04.png")
plt.show()

In [ ]:
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas
ref_data = (pd.read_csv(REF_PATH, header=None, names=['REF_PATH']))/222
inf_data = (pd.read_csv(INF_PATH, header=None, names=['INF_PATH']))/222

# Graficar histogramas
plt.hist(ref_data, bins=50, edgecolor='red', alpha=0.5, label='Reference', align='mid', color='red',density="True")
plt.hist(inf_data, bins=50, edgecolor='green', alpha=0.5, label='Inference', align='mid', color='green',density="True")

# Agregar etiquetas y título
plt.xlabel('Energy (eV)')
plt.ylabel('Density')
plt.title('Reference and inference energy distribution')

# Mostrar leyenda
plt.legend()

# Mostrar el histograma
plt.savefig("graficas-inferencias/output-graficas/figura-05.png")
plt.show()

In [ ]:
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas
ref_data = (pd.read_csv(REF_PATH, header=None, names=['REF_PATH']))/222
inf_data = (pd.read_csv(INF_PATH, header=None, names=['INF_PATH']))/222

# Calcular la diferencia absoluta entre las columnas
abs_diff = np.abs(ref_data['REF_PATH']- inf_data['INF_PATH'])

# Graficar el histograma de la diferencia absoluta
plt.hist(abs_diff, bins=50, edgecolor='black', density=True)  # Density set to True
plt.xlabel('Difference between the reference energy and the energy inferred (eV)')
plt.ylabel('Density')  # Cambiamos Frequency a Density
plt.title('Absolute energy difference distribution')
plt.grid(True)
plt.savefig("graficas-inferencias/output-graficas/figura-06.png")
plt.show()

#### Continuos

```python
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas
ref_data = pd.read_csv(REF_PATH, header=None, names=['REF_PATH']) / 222
inf_data = pd.read_csv(INF_PATH, header=None, names=['INF_PATH']) / 222

# Calcular la densidad de puntos
xy = np.vstack([ref_data['REF_PATH'], inf_data['INF_PATH']])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura
#plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura

# Graficar los puntos en 2D
plt.scatter(x, y, c=z_sorted, cmap='Blues', s=25)

# Establecer etiquetas y título
plt.colorbar(label='Density')
plt.xlabel('Energy DFT (eV)')
plt.ylabel('Energy TorchMD-Net (eV)')
plt.title('Reference energy vs Inferred energy (Density)')

# Mostrar la gráfica
plt.show()


```python
# Rutas de los archivos que se utilizarán
REF_PATH = 'graficas-inferencias/input-graficas/energy_test_sam.dat'
INF_PATH = 'graficas-inferencias/input-graficas/energy_test_inf.dat'

# Cargar los datos con Pandas
ref_data = pd.read_csv(REF_PATH, header=None, names=['REF_PATH']) / 222
inf_data = pd.read_csv(INF_PATH, header=None, names=['INF_PATH']) / 222

# Calcular la densidad de puntos
xy = np.vstack([ref_data['REF_PATH'], inf_data['INF_PATH']])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar los puntos en 3D
ax.scatter(x, y, z_sorted, c=z_sorted, cmap='Blues', s=25)

# Agregar etiquetas y título
ax.set_xlabel('DFT Energy (eV)')
ax.set_ylabel('TorchMD-Net Energy (eV)')
ax.set_zlabel('Density')
ax.set_title('Reference energy vs Inferred energy (Density Surface)')

# Mostrar la gráfica
plt.show()

## Angles

### Hydrogen

In [ ]:
# Ruta del archivo que se utilizará
Angles_H_PATH = 'graficas-inferencias/input-graficas/angles-test-1.dat'

# Cargar los datos con Pandas
angles_h_data = pd.read_csv(Angles_H_PATH, header=None, skiprows=1, delim_whitespace=True)

# Calcular el histograma
hist, bin_edges = np.histogram(angles_h_data.values.flatten(), bins=200, density=True)
bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2

# Graficar el histograma
plt.scatter(bin_centres, hist * 100)

# Definir etiquetas y títulos
plt.xlabel('Angle between DFT and TorchMD-Net forces (degrees)')
plt.ylabel('Frequency of occurrence (%)')
plt.title('Angles Distribution (Hydrogen)')

# Mostrar el gráfico
plt.ylim(0, 4)
plt.grid(True)
plt.tight_layout()
plt.savefig("graficas-inferencias/output-graficas/figura-07.png")
plt.show()

### Oxygen

In [ ]:
# Ruta del archivo que se utilizará
Angles_O_PATH = 'graficas-inferencias/input-graficas/angles-test-8.dat'

# Cargar los datos con Pandas
angles_o_data = pd.read_csv(Angles_O_PATH, header=None, skiprows=1, delim_whitespace=True)

# Calcular el histograma
hist, bin_edges = np.histogram(angles_o_data.values.flatten(), bins=200, density=True)
bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2

# Graficar el histograma
plt.scatter(bin_centres, hist * 100)

# Definir etiquetas y títulos
plt.xlabel('Angle between DFT and TorchMD-Net forces (degrees)')
plt.ylabel('Frequency of occurrence (%)')
plt.title('Angles Distribution (Oxygen)')

# Mostrar el gráfico
plt.ylim(0, 4)
plt.grid(True)
plt.tight_layout()
plt.savefig("graficas-inferencias/output-graficas/figura-08.png")
plt.show()

### Aluminium

In [ ]:
# Ruta del archivo que se utilizará
Angles_Al_PATH = 'graficas-inferencias/input-graficas/angles-test-13.dat'

# Cargar los datos con Pandas
angles_al_data = pd.read_csv(Angles_Al_PATH, header=None, skiprows=1, delim_whitespace=True)

# Calcular el histograma
hist, bin_edges = np.histogram(angles_al_data.values.flatten(), bins=200, density=True)
bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2

# Graficar el histograma
plt.scatter(bin_centres, hist * 100)

# Definir etiquetas y títulos
plt.xlabel('Angle between DFT and TorchMD-Net forces (degrees)')
plt.ylabel('Frequency of occurrence (%)')
plt.title('Angles Distribution (Aluminium)')

# Mostrar el gráfico
plt.ylim(0, 4)
plt.grid(True)
plt.tight_layout()
plt.savefig("graficas-inferencias/output-graficas/figura-09.png")
plt.show()

## Modules

### Hydrogen

In [ ]:
x_limit=12
y_limit=12

In [ ]:
# Ruta del archivo para los módulos
MODULE_H_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'

# Cargar los datos con NumPy
module_h_data = np.loadtxt(MODULE_H_PATH)

# Graficar
plt.scatter(module_h_data[:, 0], module_h_data[:, 1], s=25)
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Hydrogen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)
plt.savefig("graficas-inferencias/output-graficas/figura-10.png")
plt.show()

#### Discretos

In [ ]:
# Ruta del archivo para los módulos
MODULE_H_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'

# Cargar los datos con NumPy
module_h_data = np.loadtxt(MODULE_H_PATH)

# Cargar los datos con NumPy
ref_data_array = module_h_data[:, 0]
inf_data_array = module_h_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Filtrar los datos donde la densidad es cero
non_zero_indices = hist != 0
hist_filtered = np.where(non_zero_indices, hist, np.nan)

# Crear la gráfica
plt.imshow(hist_filtered.T, extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='Blues', origin='lower', aspect='auto')
plt.colorbar(label='Density')
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Hydrogen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-11.png")
plt.show()

In [ ]:
# Ruta del archivo para los módulos
MODULE_H_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'

# Cargar los datos con NumPy
module_h_data = np.loadtxt(MODULE_H_PATH)

# Cargar los datos con NumPy
ref_data_array = module_h_data[:, 0]
inf_data_array = module_h_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Coordenadas de las esquinas de los rectángulos
xpos, ypos = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Tamaños de los rectángulos
dx = dy = 0.04 * np.ones_like(zpos)
#dx = dy = 0.0001 * np.ones_like(zpos)
dz = hist.ravel()

# Filtrar los datos donde la densidad es cero
non_zero_indices = dz != 0
xpos = xpos[non_zero_indices]
ypos = ypos[non_zero_indices]
dz = dz[non_zero_indices]

# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar la superficie
ax.plot_trisurf(xpos, ypos, dz, cmap='Blues')

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Agregar etiquetas y título
ax.set_xlabel('DFT modules (eV/$\AA$)')
ax.set_ylabel('TorchMD-net modules (eV/$\AA$)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Hydrogen) (Surface)')

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-12.png")
plt.show()


In [ ]:
# Ruta del archivo para los módulos
MODULE_H_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'
#entrenamientos/bo222-200k-1173k/2024-03-15/graficas-inferencias/input-graficas/modules-test-1.dat

# Cargar los datos con Pandas
data = pd.read_csv(MODULE_H_PATH, header=None, delim_whitespace=True)

# Dividir los datos en dos columnas
ref_data_array = data.iloc[:, 0]  # Primera columna
inf_data_array = data.iloc[:, 1]   # Segunda columna

# Graficar histogramas
plt.hist(ref_data_array, bins=100, edgecolor='red', alpha=0.5, label='Reference', align='mid', color='red',density=True)
plt.hist(inf_data_array, bins=100, edgecolor='green', alpha=0.5, label='Inference', align='mid', color='green',density=True)

# Agregar etiquetas y título
plt.xlabel('Modules (eV/$\AA$)')
plt.ylabel('Density')
plt.title('Distribution of the module of reference and inferred forces (Hydrogen)')

# Mostrar leyenda
plt.legend()

# Mostrar el histograma
plt.savefig("graficas-inferencias/output-graficas/figura-13.png")
plt.show()

#### Continuos

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura
#plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura

# Graficar los puntos en 2D
plt.scatter(x, y, c=z_sorted, cmap='Blues', s=25)

# Establecer etiquetas y título
plt.xlabel('DFT modules (Ha/Bohr)')
plt.ylabel('TorchMD-net modules (Ha/Bohr)')
plt.title('Reference modules vs Inferred modules (Hydrogen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.colorbar(label='Density')
plt.show()

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-1.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar los puntos en 3D
ax.scatter(x, y, z_sorted, c=z_sorted, cmap='Blues', s=25)

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Establecer etiquetas y título
ax.set_xlabel('DFT modules (Ha/Bohr)')
ax.set_ylabel('TorchMD-net modules (Ha/Bohr)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Hydrogen)')

# Mostrar la gráfica
plt.show()

### Oxygen

In [ ]:
x_limit=12
y_limit=12

In [ ]:
# Ruta del archivo para los módulos
MODULE_O_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con NumPy
module_o_data = np.loadtxt(MODULE_O_PATH)

# Graficar
plt.scatter(module_o_data[:, 0], module_o_data[:, 1], s=25)
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Oxygen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)
plt.savefig("graficas-inferencias/output-graficas/figura-14.png")
plt.show()

#### Discretos

In [ ]:
# Ruta del archivo para los módulos
MODULE_O_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con NumPy
module_o_data = np.loadtxt(MODULE_O_PATH)

# Cargar los datos con NumPy
ref_data_array = module_o_data[:, 0]
inf_data_array = module_o_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Filtrar los datos donde la densidad es cero
non_zero_indices = hist != 0
hist_filtered = np.where(non_zero_indices, hist, np.nan)

# Crear la gráfica
plt.imshow(hist_filtered.T, extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='Blues', origin='lower', aspect='auto')
plt.colorbar(label='Density')
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Oxygen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-15.png")
plt.show()

In [ ]:
# Ruta del archivo para los módulos
MODULE_O_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con NumPy
module_o_data = np.loadtxt(MODULE_O_PATH)

# Cargar los datos con NumPy
ref_data_array = module_o_data[:, 0]
inf_data_array = module_o_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Coordenadas de las esquinas de los rectángulos
xpos, ypos = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Tamaños de los rectángulos
dx = dy = 0.04 * np.ones_like(zpos)
#dx = dy = 0.0001 * np.ones_like(zpos)
dz = hist.ravel()

# Filtrar los datos donde la densidad es cero
non_zero_indices = dz != 0
xpos = xpos[non_zero_indices]
ypos = ypos[non_zero_indices]
dz = dz[non_zero_indices]

# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar la superficie
ax.plot_trisurf(xpos, ypos, dz, cmap='Blues')

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Agregar etiquetas y título
ax.set_xlabel('DFT modules (eV/$\AA$)')
ax.set_ylabel('TorchMD-net modules (eV/$\AA$)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Oxygen) (Surface)')

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-16.png")
plt.show()

In [ ]:
# Ruta del archivo para los módulos
MODULE_O_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con Pandas
data = pd.read_csv(MODULE_O_PATH, header=None, delim_whitespace=True)

# Dividir los datos en dos columnas
ref_data_array = data.iloc[:, 0]  # Primera columna
inf_data_array = data.iloc[:, 1]   # Segunda columna

# Graficar histogramas
plt.hist(ref_data_array, bins=100, edgecolor='red', alpha=0.5, label='Reference', align='mid', color='red',density="True")
plt.hist(inf_data_array, bins=100, edgecolor='green', alpha=0.5, label='Inference', align='mid', color='green',density="True")

# Agregar etiquetas y título
plt.xlabel('Modules (eV/$\AA$)')
plt.ylabel('Density')
plt.title('Distribution of the module of reference and inferred forces (Oxygen)')

# Mostrar leyenda
plt.legend()

# Mostrar el histograma
plt.savefig("graficas-inferencias/output-graficas/figura-17.png")
plt.show()

#### Continuos

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura
#plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura

# Graficar los puntos en 2D
plt.scatter(x, y, c=z_sorted, cmap='Blues', s=25)

# Establecer etiquetas y título
plt.xlabel('DFT modules (Ha/Bohr)')
plt.ylabel('TorchMD-net modules (Ha/Bohr)')
plt.title('Reference modules vs Inferred modules (Oxygen)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.colorbar(label='Density')
plt.show()

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-8.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar los puntos en 3D
ax.scatter(x, y, z_sorted, c=z_sorted, cmap='Blues', s=25)

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Establecer etiquetas y título
ax.set_xlabel('DFT modules (Ha/Bohr)')
ax.set_ylabel('TorchMD-net modules (Ha/Bohr)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Oxygen)')

# Mostrar la gráfica
plt.show()

### Aluminium

In [ ]:
x_limit=12
y_limit=12

In [ ]:
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con NumPy
module_al_data = np.loadtxt(MODULE_Al_PATH)

# Graficar
plt.scatter(module_al_data[:, 0], module_al_data[:, 1], s=25)
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Aluminium)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)
plt.savefig("graficas-inferencias/output-graficas/figura-18.png")
plt.show()

#### Discretos

In [ ]:
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con NumPy
module_al_data = np.loadtxt(MODULE_Al_PATH)

# Cargar los datos con NumPy
ref_data_array = module_al_data[:, 0]
inf_data_array = module_al_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Filtrar los datos donde la densidad es cero
non_zero_indices = hist != 0
hist_filtered = np.where(non_zero_indices, hist, np.nan)

# Crear la gráfica
plt.imshow(hist_filtered.T, extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='Blues', origin='lower', aspect='auto')
plt.colorbar(label='Density')
plt.xlabel('DFT modules (eV/$\AA$)')
plt.ylabel('TorchMD-net modules (eV/$\AA$)')
plt.title('Reference modules vs Inferred modules (Aluminium)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-19.png")
plt.show()

In [ ]:
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con NumPy
module_al_data = np.loadtxt(MODULE_Al_PATH)

# Cargar los datos con NumPy
ref_data_array = module_al_data[:, 0]
inf_data_array = module_al_data[:, 1]

# Calcular el histograma 2D
hist, xedges, yedges = np.histogram2d(ref_data_array, inf_data_array, bins=50, density=True)

# Coordenadas de las esquinas de los rectángulos
xpos, ypos = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Tamaños de los rectángulos
dx = dy = 0.04 * np.ones_like(zpos)
#dx = dy = 0.0001 * np.ones_like(zpos)
dz = hist.ravel()

# Filtrar los datos donde la densidad es cero
non_zero_indices = dz != 0
xpos = xpos[non_zero_indices]
ypos = ypos[non_zero_indices]
dz = dz[non_zero_indices]

# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar la superficie
ax.plot_trisurf(xpos, ypos, dz, cmap='Blues')

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Agregar etiquetas y título
ax.set_xlabel('DFT modules (eV/$\AA$)')
ax.set_ylabel('TorchMD-net modules (eV/$\AA$)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Aluminium) (Surface)')

# Mostrar la gráfica
plt.savefig("graficas-inferencias/output-graficas/figura-20.png")
plt.show()

In [ ]:
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con Pandas
data = pd.read_csv(MODULE_Al_PATH, header=None, delim_whitespace=True)

# Dividir los datos en dos columnas
ref_data_array = data.iloc[:, 0]  # Primera columna
inf_data_array = data.iloc[:, 1]  # Segunda columna

# Graficar histogramas
plt.hist(ref_data_array, bins=100, edgecolor='red', alpha=0.5, label='Reference', align='mid', color='red',density="True")
plt.hist(inf_data_array, bins=100, edgecolor='green', alpha=0.5, label='Inference', align='mid', color='green',density="True")

# Agregar etiquetas y título
plt.xlabel('Modules (eV/$\AA$)')
plt.ylabel('Density')
plt.title('Distribution of the module of reference and inferred forces (Aluminium)')

# Mostrar leyenda
plt.legend()

# Mostrar el histograma
plt.savefig("graficas-inferencias/output-graficas/figura-21.png")
plt.show()

#### Continuos

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura
#plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura

# Graficar los puntos en 2D
plt.scatter(x, y, c=z_sorted, cmap='Blues', s=25)

# Establecer etiquetas y título
plt.xlabel('DFT modules (Ha/Bohr)')
plt.ylabel('TorchMD-net modules (Ha/Bohr)')
plt.title('Reference modules vs Inferred modules (Aluminium)')
plt.xlim(0, x_limit)
plt.ylim(0, y_limit)

# Mostrar la gráfica
plt.colorbar(label='Density')
plt.show()

```python
# Ruta del archivo para los módulos
MODULE_Al_PATH = 'graficas-inferencias/input-graficas/modules-test-13.dat'

# Cargar los datos con NumPy
data = np.loadtxt(MODULE_Al_PATH)

# Dividir los datos en dos columnas
ref_data_array = data[:, 0]  # Primera columna
inf_data_array = data[:, 1]  # Segunda columna

# Calcular la densidad de puntos
xy = np.vstack([ref_data_array, inf_data_array])
z = gaussian_kde(xy)(xy)

# Ordenar los puntos por densidad, para que los puntos más densos se tracen al final
idx = z.argsort()
xy_sorted = xy[:, idx]
z_sorted = z[idx]

# Asignar los valores ordenados de x e y
x, y = xy_sorted

```python
# Crear una figura y obtener ejes 3D
fig = plt.figure(figsize=(10, 8))  # Ajustar el tamaño de la figura
ax = fig.add_subplot(111, projection='3d')

# Graficar los puntos en 3D
ax.scatter(x, y, z_sorted, c=z_sorted, cmap='Blues', s=25)

# Establecer los límites de los ejes x y y
ax.set_xlim(0, x_limit)
ax.set_ylim(0, y_limit)

# Establecer etiquetas y título
ax.set_xlabel('DFT modules (Ha/Bohr)')
ax.set_ylabel('TorchMD-net modules (Ha/Bohr)')
ax.set_zlabel('Density')
ax.set_title('Reference modules vs Inferred modules (Aluminium)')

# Mostrar la gráfica
plt.show()